# Translation from English to Spanish using Flan-T5 and Helsinki-NLP/opus-100 Dataset


## Introduction
In this notebook, we will use the Flan-T5 model to perform translation from English to Spanish. We will use the "Helsinki-NLP/opus-100" dataset from Hugging Face, specifically the en-es subset, to train and evaluate our translation model.


In [1]:
#!pip install transformers tensorflow datasets

In [1]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", from_pt=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
/usr/local/lib/python3.12/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected

## Loading the Dataset

In [2]:
from datasets import load_dataset

# Load the Helsinki-NLP/opus-100 dataset
dataset = load_dataset('Helsinki-NLP/opus-100', 'en-es')
print(dataset['train'][0])


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/237k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/238k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': "It was the asbestos in here, that's what did it!", 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}}


## Data Preprocessing

In [7]:
# Preprocess the dataset for input into the model
def preprocess_data(examples):
    inputs = [f'Translate from English to Spanish: {example["en"]}' for example in examples['translation']]
    targets = [example['es'] for example in examples['translation']]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]

    # For decoder inputs
    decoder_inputs = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["decoder_input_ids"] = decoder_inputs["input_ids"]

    return model_inputs

# Apply preprocessing to the dataset
train_dataset_ = dataset['train'].select(range(30000)).map(preprocess_data, batched=True) # use only 30000 examples
test_dataset_ = dataset['test'].map(preprocess_data, batched=True)

print(train_dataset_[0])


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': "It was the asbestos in here, that's what did it!", 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}, 'input_ids': [30355, 15, 45, 1566, 12, 5093, 10, 94, 47, 8, 23778, 16, 270, 6, 24, 31, 7, 125, 410, 34, 55, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [6343, 49, 106, 10381

## Converting to TensorFlow Datasets

In [8]:
import tensorflow as tf

# Convert Hugging Face datasets to TensorFlow datasets
train_dataset = train_dataset_.to_tf_dataset( # object.method()
    columns=['input_ids', 'attention_mask', 'decoder_input_ids'],
    label_cols=['labels'], #what I'm comparing against
    shuffle=True,
    batch_size=64, # I changed from 64 to 16 to use less memory
    collate_fn=None # If I have the final batch, do nothing
)

test_dataset = test_dataset_.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'decoder_input_ids'],
    label_cols=['labels'],
    shuffle=False,
    batch_size=64, # I changed from 64 to 16 to use less memory
    collate_fn=None
)


/usr/local/lib/python3.12/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


## Freezing the Model

In [9]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247577856 (944.43 MB)
Trainable params: 247577856 (944.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Freeze the LLM layer, we train the final head layer
model.get_layer("shared").trainable = False
model.get_layer("encoder").trainable = False
model.get_layer("decoder").trainable = False


In [11]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247577856 (944.43 MB)
Trainable params: 24674304 (94.12 MB)
Non-trainable params: 222903552 (850.31 MB)
_________________________________________________________________



### Important Considerations in Transfer Learning

1. **Freezing the LLM Layer:** In transfer learning, it's important to freeze the pre-trained language model layer to retain the knowledge it has already acquired and to avoid overfitting. This allows the model to leverage its pre-trained capabilities while focusing on learning the new task-specific nuances.

2. **Loss Function with `from_logits=True`:** When fine-tuning language models from Hugging Face, it's crucial to use the loss function with `from_logits=True`. This is because these models do not apply softmax to their outputs, and using `from_logits=True` ensures that the loss is computed correctly.


## Model Training

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
  # SparseCategoricalCrossentropy and from_logits=True are essential for Hugging face

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=3)



Epoch 1/3
469/469 [==============================] - 798s 2s/step - loss: 25.6789 - val_loss: 2.5515
Epoch 2/3
469/469 [==============================] - 763s 2s/step - loss: 1.4609 - val_loss: 0.8412
Epoch 3/3
469/469 [==============================] - 763s 2s/step - loss: 0.7799 - val_loss: 0.6225


- Check if the Loss decreases at each epoch
- check validation loss went down and didn't spike, which is good because we are avoiding overfitting



## Performing Translation

In [ ]:
# Perform translation on a few examples from the test set
def translate(inputs):
    outputs = model.generate(inputs[0]["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Translate and display a few examples
for batch in test_dataset.take(5):
    translated_text = translate(batch)
    print(f"Input: {tokenizer.decode(batch[0]['input_ids'][0], skip_special_tokens=True)}")
    print(f"Reference Translation: {tokenizer.decode(batch[1][0], skip_special_tokens=True)}")
    print(f"Translated Text: {translated_text}")
    print()



## Conclusion
In this notebook, we used the Flan-T5 model to perform translation from English to Spanish using the Helsinki-NLP/opus-100 dataset. We preprocessed the dataset, fine-tuned the model while freezing the LLM layer, and performed translations. We manually validated the translations to assess the quality of the model's performance. The results demonstrate the effectiveness of the Flan-T5 model for translation tasks.
